# Convert PyTorch to ONNX to TFLite

### RAW CNN Conversion

In [2]:
import sys
sys.path.append("D:\Programming\cybertruck")

from cnn.raw_cnn import Raw_CNN
import torch.onnx

In [5]:
raw_model = Raw_CNN(None).to('cuda')
x = torch.randn(5, 3, 229, 229).to('cuda')
raw_model.load_state_dict(torch.load(r"D:\Programming\cybertruck\weights\raw\epoch20_11-27_16_15_10_76acc.pt"))


<All keys matched successfully>

In [6]:
output = raw_model(x)

In [7]:
output.shape

torch.Size([5, 10])

In [8]:
torch.onnx.export(raw_model, 
                  x, 
                  'raw_cnn.onnx',
                  export_params=True, 
                  opset_version=17, 
                  do_constant_folding=True, 
                  input_names = ['input'], 
                  output_names = ['output'], 
                  dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})


### Now Prepare the Onnx model for tensorflow and then convert to tflite

https://medium.com/@zergtant/convert-pytorch-model-to-tf-lite-with-onnx-tf-232a3894657c

In [15]:
import onnx_tf
import onnx

# Load  ONNX model
onnx_model = onnx.load('raw_cnn.onnx')

# Convert ONNX model to TensorFlow format
tf_model = onnx_tf.backend.prepare(onnx_model)
# Export  TensorFlow  model 
tf_model.export_graph("raw_cnn.tf")

c:\Developer\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Developer\Python310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_230_x, add_43_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: raw_cnn.tf\assets


INFO:tensorflow:Assets written to: raw_cnn.tf\assets
INFO:absl:Writing fingerprint to raw_cnn.tf\fingerprint.pb


In [16]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("raw_cnn.tf")
tflite_model = converter.convert()
open('raw_cnn.tflite', 'wb').write(tflite_model)

83265428

### Now test the converted model

https://www.tensorflow.org/lite/guide/inference
https://www.tensorflow.org/lite/guide/inference#run_inference_with_dynamic_shape_model

Note that by default, this model takes a batch size of 1. Which works out well since we want real time performance

In [28]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="raw_cnn.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[  1   3 229 229]
[[ 0.70486337  1.1734309  -0.31111112 -0.14354168 -1.9417019  -0.6325212
  -0.0252881   0.5548716   1.183073   -0.6819163 ]]


In [30]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_path="raw_cnn.tflite")

# Resize input shape for dynamic shape model and allocate tensor
interpreter.resize_tensor_input(interpreter.get_input_details()[0]['index'], [5, 3, 229, 229])
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details) # Here we can see the input size has changed to (5, 3, 229, 229) 
print(output_details) # The output has the corresponding changes

input_data = np.array(np.random.random_sample((5, 3, 229, 229)), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  5,   3, 229, 229]), 'shape_signature': array([ -1,   3, 229, 229]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'PartitionedCall:0', 'index': 360, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[ 0.67858803  1.2608714  -0.35024315 -0.16926882 -2.035589   -0.63033855
  -0.02604075  0.5862357   1.2443491  -0.75167644]
 [ 0.697458    1.2371234  -0.23860058 -0.18017717 -2.0142417  -0.73387134
   0.0772187   0.56821257  1.2372719  -0.7376198 ]
 [ 0.6675122   1.1572503  -0.30434915 -0.11979293 -1.9236575  -0.6253245